In [ ]:
import sqlite3

db_path = './dso_data.db'
conn = sqlite3.connect(db_path)
c = conn.cursor()

select_query = "SELECT * FROM dso WHERE catalog LIKE 'M %' LIMIT 5;"
c.execute(select_query)
rows = c.fetchall()

print("Messier Objects:")
for row in rows:
    print(row)  

conn.close()



In [ ]:
import datetime
from zoneinfo import ZoneInfo

conn = sqlite3.connect(db_path)
c = conn.cursor()
cursor = conn.cursor()
cursor.execute("DROP TABLE IF EXISTS test;")
cursor.execute("Create table test (transit TEXT, rise TEXT, setting TEXT);")
insert_query = "INSERT INTO test (transit) VALUES (?);"

now = datetime.datetime.now().astimezone(ZoneInfo("America/Chicago"))
value = now.isoformat()
print(f"inserting: {value}")
cursor.execute(insert_query, (value,))
conn.commit()
cursor.execute("SELECT DATETIME(transit), DATE(transit), TIME(transit, 'localtime') FROM test;")
rows = cursor.fetchall()
print("\nTable Data:")
for row in rows:
    print(row, f"dt: {row[0]}, date: {row[1]}, time: {row[2]}" )  
# c.execute("DROP TABLE IF EXISTS test;")
conn.close()

In [ ]:
import datetime
from zoneinfo import ZoneInfo
import sqlite3

db_path = './dso_data.db'

conn = sqlite3.connect(db_path)
c = conn.cursor()
cursor = conn.cursor()
cursor.execute("DROP TABLE IF EXISTS test;")
cursor.execute("Create table test (transit TEXT, rise TEXT, setting TEXT);")
insert_query = "INSERT INTO test (transit) VALUES (?);"

today_local = datetime.datetime.now(ZoneInfo("America/Chicago")).replace(hour=0, minute=0, second=0, microsecond=0)
# print(f"Local midnight: {today_local.isoformat()}")
# tenPM_local = today_local.replace(hour=22, minute=0, second=0, microsecond=0)
# tenPM_utc = tenPM_local.astimezone(ZoneInfo("UTC"))
# print(f"Local time in ISO: {tenPM_local.isoformat()}")
# print(f"Local time {tenPM_local} in UTC: {tenPM_utc.isoformat()}")

# transit at 10PM Local time today
transit_local = today_local.replace(hour=23, minute=0, second=0, microsecond=0)
transit_utc = transit_local.astimezone(ZoneInfo("UTC"))
value = transit_utc.strftime('%Y-%m-%d %H:%M:%S')
print(f"inserting transit {transit_local} as utc iso: {value}")
cursor.execute(insert_query, (value,))
conn.commit()
cursor.execute("insert into test (transit) values (datetime('now'));")
conn.commit()


cursor.execute("SELECT DATETIME(transit), DATETIME(transit,'utc'),  DATE(transit, 'localtime'), TIME(transit, 'localtime'), strftime('%Y-%m-%d %H:%M:%S', transit), transit FROM test;")
rows = cursor.fetchall()
print("\nTable Data:")
for row in rows:
    print(row, f"dt: {row[0]}, dt-utc: {row[1]}, date: {row[2]}, time: {row[3]}, strp: {row[4]}, raw:{row[5]}" )  
# c.execute("DROP TABLE IF EXISTS test;")

local_start = today_local.replace(hour=18, minute=0, second=0, microsecond=0)
local_end = today_local.replace(hour=2, minute=0, second=0, microsecond=0) + datetime.timedelta(days=1)

local_start_utc = local_start.astimezone(ZoneInfo('UTC'))
local_end_utc = local_end.astimezone(ZoneInfo('UTC'))

print(f"Local start: {local_start} -> {local_start_utc.strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Local end: {local_end} -> {local_end_utc.strftime('%Y-%m-%d %H:%M:%S')}")

query = f"SELECT count(*) from test WHERE transit > '{local_start_utc.strftime('%Y-%m-%d %H:%M:%S')}' and transit < '{local_end_utc.strftime('%Y-%m-%d %H:%M:%S')}';"
print(f"Executing UTC based Count Query: {query}")
cursor.execute(query)
rows = cursor.fetchall()
print("\nUTC based Count Query Result:")
for row in rows:
    print(row)      

# convert all times to utc, then format as strftime('%Y-%m-%dT%H:%M:%SZ') for storage
# when querying for local day ranges, convert local start and end to utc, then query using those utc times and format
# the formats must exactly match for lexical comparison to work correctly

conn.close()

inserting transit 2025-12-04 23:00:00-06:00 as utc iso: 2025-12-05 05:00:00

Table Data:
('2025-12-05 05:00:00', '2025-12-05 11:00:00', '2025-12-04', '23:00:00', '2025-12-05 05:00:00', '2025-12-05 05:00:00') dt: 2025-12-05 05:00:00, dt-utc: 2025-12-05 11:00:00, date: 2025-12-04, time: 23:00:00, strp: 2025-12-05 05:00:00, raw:2025-12-05 05:00:00
('2025-12-05 02:03:49', '2025-12-05 08:03:49', '2025-12-04', '20:03:49', '2025-12-05 02:03:49', '2025-12-05 02:03:49') dt: 2025-12-05 02:03:49, dt-utc: 2025-12-05 08:03:49, date: 2025-12-04, time: 20:03:49, strp: 2025-12-05 02:03:49, raw:2025-12-05 02:03:49
Local start: 2025-12-04 18:00:00-06:00 -> 2025-12-05 00:00:00
Local end: 2025-12-05 02:00:00-06:00 -> 2025-12-05 08:00:00
Executing UTC based Count Query: SELECT count(*) from test WHERE transit > '2025-12-05 00:00:00' and transit < '2025-12-05 08:00:00';

UTC based Count Query Result:
(2,)
